In [51]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score
from helpers import *
# Load the TSV file
df = pd.read_csv('ml-32m/ratings.csv')


# Print the head of the DataFrame
print(df.head())

   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40


In [8]:
data_folder = './MovieSummaries/'
#paths to files
plot_summaries_path = data_folder + 'plot_summaries.txt'
movie_metadata_path = data_folder + 'movie.metadata.tsv'
character_metadata_path = data_folder + 'character.metadata.tsv'

# load the data
# 1. Plot summaries data
plot_summaries_df = pd.read_csv(plot_summaries_path, delimiter='\t', names=['wikipedia_movie_id', 'plot_summary'], 
                                 encoding='utf-8')

# 2. Movie metadata
movie_metadata_df = pd.read_csv(movie_metadata_path, delimiter='\t', names=['wikipedia_movie_id', 'freebase_movie_id', 
                                                                            'movie_name', 'release_date', 'box_office_revenue',
                                                                            'runtime', 'languages', 'countries', 'genres'], 
                                 encoding='utf-8')

# 3. Character metadata
character_metadata_df = pd.read_csv(character_metadata_path, delimiter='\t', names=['wikipedia_movie_id', 'freebase_movie_id', 'release_date', 'character_name', 
                                                                                    'actor_dob', 'actor_gender', 'actor_height', 'actor_ethnicity', 'actor_name', 
                                                                                    'actor_age_at_release', 'freebase_character_actor_map_id', 'freebase_character_id', 
                                                                                    'freebase_actor_id'], 
                                     encoding='utf-8')


In [9]:
movies = pd.read_csv('ml-32m/movies.csv')

In [10]:
movie_metadata_df['movie_name_formatted'] = movie_metadata_df['movie_name'].str.lower().str.strip()
movies['title_format'] = movies['title'].str[:-6].str.strip().str.lower()


In [11]:
movies.head()

,movieId,title,genres,title_format
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story
1,2,Jumanji (1995),Adventure|Children|Fantasy,jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,grumpier old men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,waiting to exhale
4,5,Father of the Bride Part II (1995),Comedy,father of the bride part ii


In [12]:
# calculate the number of common movies between the two datasets

common_movies = set(movie_metadata_df['movie_name_formatted']).intersection(set(movies['title_format']))
print('Number of common movies:', len(common_movies))

# merge the two datasets

merged_df = pd.merge(movies, movie_metadata_df, left_on='title_format', right_on='movie_name_formatted', how='inner')

Number of common movies: 12318


In [13]:
merged_df

,movieId,title,genres_x,title_format,wikipedia_movie_id,freebase_movie_id,movie_name,release_date,box_office_revenue,runtime,languages,countries,genres_y,movie_name_formatted
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,53085,/m/0dyb1,Toy Story,1995-11-19,361958736.0,77.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0556j8"": ""Buddy film"", ""/m/03k9fj"": ""Adve...",toy story
1,2,Jumanji (1995),Adventure|Children|Fantasy,jumanji,3700174,/m/09w353,Jumanji,1995-12-15,262797249.0,104.0,"{""/m/064_8sq"": ""French Language"", ""/m/02h40lc""...","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/0hj3n2k"": ""Fanta...",jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,grumpier old men,1934035,/m/0676dr,Grumpier Old Men,1995-12-22,71518503.0,101.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/0556j8"": ""...",grumpier old men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,waiting to exhale,972970,/m/03vny7,Waiting to Exhale,1995-12-22,81452156.0,121.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3n0w"": ""Ensemble Film"", ""/m/06w2n3t"": ...",waiting to exhale
4,5,Father of the Bride Part II (1995),Comedy,father of the bride part ii,3303622,/m/094g2z,Father of the Bride Part II,1995-12-08,76594107.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06qm3"": ""Screwball comedy"", ""/m/02l7c8"": ...",father of the bride part ii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17557,131152,The Fat Spy (1966),Comedy,the fat spy,1853200,/m/0613sx,The Fat Spy,1966,NaN,75.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0gf28"": ""Parody"", ""/m/0220p9g"": ""Musical ...",the fat spy
17558,131166,WWII IN HD (2009),(no genres listed),wwii in hd,26321497,/m/0bbxzy2,WWII in HD,NaN,NaN,NaN,{},{},"{""/m/082gq"": ""War film""}",wwii in hd
17559,131239,Three Quarter Moon (2011),Comedy|Drama,three quarter moon,33597212,/m/0hgrd5g,Three Quarter Moon,2011-09-30,NaN,95.0,"{""/m/02hwyss"": ""Turkish Language"", ""/m/0gtg"": ...","{""/m/0345h"": ""Germany""}","{""/m/0556j8"": ""Buddy film"", ""/m/0hqxf"": ""Famil...",three quarter moon
17560,131248,Brother Bear 2 (2006),Adventure|Animation|Children|Comedy|Fantasy,brother bear 2,4118248,/m/0bk2k2,Brother Bear 2,2006-08-29,NaN,73.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hqxf"": ""Family Film"", ""/m/0hcr"": ""Animat...",brother bear 2


In [14]:
# Remove this user from the dataset, because it has too many ratings
df = df[df['userId'] != 175325]

In [15]:
list_of_genres=movies['genres'].unique()

In [16]:
# Print the user with the most ratings
user_ratings_count = df['userId'].value_counts()

print('User with the most ratings:')
print(user_ratings_count.head(20))


User with the most ratings:
118205    9254
8405      7515
82418     5646
121535    5520
125794    5491
74142     5447
34576     5356
131904    5330
83090     5169
59477     4988
130767    4785
79159     4707
8963      4524
15617     4354
92011     4236
71975     4182
20132     4101
46470     4094
88820     4093
63147     3958
Name: userId, dtype: int64


In [17]:
rating_films=pd.merge(movies,df,on='movieId',how='inner')


In [18]:
unique_genres = movies['genres'].unique()
all_genres = set('|'.join(unique_genres).split('|'))

all_genres_list = list(all_genres)
all_genres_list.remove('(no genres listed)')
print(all_genres_list)


['War', 'Action', 'Children', 'Fantasy', 'Musical', 'Mystery', 'Film-Noir', 'IMAX', 'Comedy', 'Animation', 'Sci-Fi', 'Romance', 'Horror', 'Drama', 'Adventure', 'Crime', 'Western', 'Thriller', 'Documentary']


In [19]:
# Create a one hot encoding for each genre
for genre in all_genres_list : 
    rating_films[genre] = rating_films['genres'].apply(lambda x: genre in x.split('|'))
rating_films

,movieId,title,genres,title_format,userId,rating,timestamp,War,Action,Children,...,Animation,Sci-Fi,Romance,Horror,Drama,Adventure,Crime,Western,Thriller,Documentary
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,3,4.0,1999-12-11 13:36:47,False,False,True,...,True,False,False,False,False,True,False,False,False,False
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,6,5.0,1997-03-13 17:50:52,False,False,True,...,True,False,False,False,False,True,False,False,False,False
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,8,4.0,1996-06-05 13:37:51,False,False,True,...,True,False,False,False,False,True,False,False,False,False
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,10,4.0,1999-11-25 02:44:47,False,False,True,...,True,False,False,False,False,True,False,False,False,False
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,11,4.5,2009-01-02 01:13:41,False,False,True,...,True,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20000258,131254,Kein Bund für's Leben (2007),Comedy,kein bund für's leben,79570,4.0,2015-03-30 19:32:59,False,False,False,...,False,False,False,False,False,False,False,False,False,False
20000259,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,"feuer, eis & dosenbier",79570,4.0,2015-03-30 19:48:08,False,False,False,...,False,False,False,False,False,False,False,False,False,False
20000260,131258,The Pirates (2014),Adventure,the pirates,28906,2.5,2015-03-30 19:56:32,False,False,False,...,False,False,False,False,False,True,False,False,False,False
20000261,131260,Rentun Ruusu (2001),(no genres listed),rentun ruusu,65409,3.0,2015-03-30 19:57:46,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
def rating_for_userID_generator(userId):
    user_rating = rating_films[rating_films['userId'] == userId]
    user_rating = user_rating.iloc[:, 5:].sample(frac=1)  
    user_rating = user_rating.drop(columns=['timestamp'])
    y = user_rating['rating']
    x = user_rating.drop(columns=['rating'])
    
    length = len(y)
    x_size = int(length * 0.8) 
    
    x_train = x.iloc[:x_size, :]
    x_test = x.iloc[x_size:, :]
    y_train = y.iloc[:x_size]
    y_test = y.iloc[x_size:]
    
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = rating_for_userID_generator(8405)


In [49]:


# Define the neural network model
model = Sequential([
    Dense(64, input_dim=x_train.shape[1], activation='relu'),  # First hidden layer
    Dense(32, activation='relu'),  # Second hidden layer
    Dense(1, activation='linear')  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Train the model
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Predict on both training and test sets
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# Evaluate the model
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Training Set Evaluation:")
print("MSE:", train_mse)
print("R^2 Score:", train_r2)

print("\nTest Set Evaluation:")
print("MSE:", test_mse)
print("R^2 Score:", test_r2)



Epoch 1/100
151/151 [==============================] - 1s 3ms/step - loss: 2.9080 - mse: 2.9080 - val_loss: 0.8932 - val_mse: 0.8932
Epoch 2/100
151/151 [==============================] - 0s 2ms/step - loss: 0.8242 - mse: 0.8242 - val_loss: 0.8123 - val_mse: 0.8123
Epoch 3/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7725 - mse: 0.7725 - val_loss: 0.8058 - val_mse: 0.8058
Epoch 4/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7607 - mse: 0.7607 - val_loss: 0.7855 - val_mse: 0.7855
Epoch 5/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7535 - mse: 0.7535 - val_loss: 0.7810 - val_mse: 0.7810
Epoch 6/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7497 - mse: 0.7497 - val_loss: 0.7902 - val_mse: 0.7902
Epoch 7/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7424 - mse: 0.7424 - val_loss: 0.7871 - val_mse: 0.7871
Epoch 8/100
151/151 [==============================] - 0s 2ms/step - 

In [34]:


def evaluate_model_for_user(userId, data):
    """
    Generates training and test splits for a given user ID, trains a neural network, 
    and evaluates the model to return MSE and R^2 for both training and test sets.
    """
    # Filter and preprocess data for the given user
    user_rating = data[data['userId'] == userId]
    user_rating = user_rating.iloc[:, 5:].sample(frac=1)  # Shuffle data
    user_rating = user_rating.drop(columns=['timestamp'])
    y = user_rating['rating']
    x = user_rating.drop(columns=['rating'])

    # Split into training and testing sets
    length = len(y)
    x_size = int(length * 0.8)
    x_train, x_test = x.iloc[:x_size, :], x.iloc[x_size:, :]
    y_train, y_test = y.iloc[:x_size], y.iloc[x_size:]

    # Define the neural network model
    model = Sequential([
        Dense(64, input_dim=x_train.shape[1], activation='relu'),  # First hidden layer
        Dense(32, activation='relu'),  # Second hidden layer
        Dense(1, activation='linear')  # Output layer for regression
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    # Train the model
    model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

    # Predict on training and test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Calculate evaluation metrics
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # Print evaluation results
    print(f"Evaluation for User ID {userId}:")
    print("Training Set Evaluation:")
    print(f"MSE: {train_mse:.4f}, R^2: {train_r2:.4f}")
    print("\nTest Set Evaluation:")
    print(f"MSE: {test_mse:.4f}, R^2: {test_r2:.4f}")

    # Return MSE values for both sets
    return train_mse, test_mse

train_mse, test_mse = evaluate_model_for_user(8405, rating_films)


Epoch 1/100
151/151 [==============================] - 1s 4ms/step - loss: 2.9541 - mse: 2.9541 - val_loss: 0.8768 - val_mse: 0.8768
Epoch 2/100
151/151 [==============================] - 0s 2ms/step - loss: 0.8623 - mse: 0.8623 - val_loss: 0.7915 - val_mse: 0.7915
Epoch 3/100
151/151 [==============================] - 0s 3ms/step - loss: 0.7977 - mse: 0.7977 - val_loss: 0.7790 - val_mse: 0.7790
Epoch 4/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7797 - mse: 0.7797 - val_loss: 0.7751 - val_mse: 0.7751
Epoch 5/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7723 - mse: 0.7723 - val_loss: 0.7774 - val_mse: 0.7774
Epoch 6/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7656 - mse: 0.7656 - val_loss: 0.7875 - val_mse: 0.7875
Epoch 7/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7626 - mse: 0.7626 - val_loss: 0.7829 - val_mse: 0.7829
Epoch 8/100
151/151 [==============================] - 0s 2ms/step - 

In [23]:
ratings_model_df=one_hot_encoding_genre(df,movies)

In [50]:
Total_eval(8405,df,movies)

Epoch 1/100
151/151 [==============================] - 1s 3ms/step - loss: 3.6454 - mse: 3.6454 - val_loss: 0.8604 - val_mse: 0.8604
Epoch 2/100
151/151 [==============================] - 0s 2ms/step - loss: 0.8248 - mse: 0.8248 - val_loss: 0.7681 - val_mse: 0.7681
Epoch 3/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7766 - mse: 0.7766 - val_loss: 0.7500 - val_mse: 0.7500
Epoch 4/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7628 - mse: 0.7628 - val_loss: 0.7596 - val_mse: 0.7596
Epoch 5/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7569 - mse: 0.7569 - val_loss: 0.7454 - val_mse: 0.7454
Epoch 6/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7523 - mse: 0.7523 - val_loss: 0.7496 - val_mse: 0.7496
Epoch 7/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7472 - mse: 0.7472 - val_loss: 0.7654 - val_mse: 0.7654
Epoch 8/100
151/151 [==============================] - 0s 2ms/step - 

(0.7025416479023149, 0.8375643576717171)

## Adding data 

In [69]:
df_tmdb_movie = pd.read_csv('TMdB_Movielens_combined/movies_metadata.csv', low_memory=False)
df_tmdb_credits = pd.read_csv('TMdB_Movielens_combined/credits.csv')
df_tmdb_keywords = pd.read_csv('TMdB_Movielens_combined/keywords.csv')

In [57]:
#director, length, vote_average, release_date
df_tmdb_keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [75]:
df_tmdb_movie.drop_duplicates(subset=['id'], keep='first', inplace=True, ignore_index=True)
df_tmdb_credits.drop_duplicates(subset=['id'], keep='first', inplace=True, ignore_index=True)
df_tmdb_keywords.drop_duplicates(subset=['id'], keep='first', inplace=True, ignore_index=True)
# To merge we need to have a common type 
df_tmdb_movie['id'] = pd.to_numeric(df_tmdb_movie['id'], errors='coerce')
df_tmdb_credits['id'] = pd.to_numeric(df_tmdb_credits['id'], errors='coerce')
df_tmdb_keywords['id'] = pd.to_numeric(df_tmdb_keywords['id'], errors='coerce')
df_tmdb_movie.dropna(subset=['id'], inplace=True)
df_tmdb_credits.dropna(subset=['id'], inplace=True)
df_tmdb_keywords.dropna(subset=['id'], inplace=True)
df_tmp = pd.merge(df_tmdb_movie, df_tmdb_credits, on='id', how='inner')
df_tmp = df_tmp.rename(columns={"original_title": "title_format"})
df_tmp['title_format']=df_tmp['title_format'].apply(
    lambda x : x.lower()
)
df_tmp.head(2)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,title_format,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,toy story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844.0,tt0113497,en,jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


In [106]:
selected_columns = df_tmp[['title_format', 'runtime', 'crew']]
selected_columns['director'] = selected_columns['crew'].apply(get_director)
selected_columns.drop(columns='crew', inplace=True)
test_merge=pd.merge(selected_columns,movies, on='title_format',how='inner')
test_merge[test_merge['director'].isna()]='nobody'
director_counts = test_merge['director'].value_counts()
test_merge['director'] = test_merge['director'].apply(
    lambda x: x if director_counts[x] > 10 else 'nobody'
)
test_merge.drop(columns=['title_format','title'], inplace=True)
test_merge

/var/folders/bz/jj60x6y164ldxfdjhqsnv6dm0000gn/T/ipykernel_48654/3334071302.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['director'] = selected_columns['crew'].apply(get_director)
/var/folders/bz/jj60x6y164ldxfdjhqsnv6dm0000gn/T/ipykernel_48654/3334071302.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns.drop(columns='crew', inplace=True)


,runtime,director,movieId,genres
0,81.0,nobody,1,Adventure|Animation|Children|Comedy|Fantasy
1,104.0,nobody,2,Adventure|Children|Fantasy
2,101.0,nobody,3,Comedy|Romance
3,127.0,nobody,4,Comedy|Drama|Romance
4,106.0,nobody,5,Comedy
...,...,...,...,...
18617,705.0,nobody,7502,Action|Drama|War
18618,60.0,nobody,27627,Drama|Romance
18619,89.0,nobody,121821,Drama|Romance
18620,143.0,nobody,96704,Drama


In [115]:
def Total_eval_test(userId,df,movies) :
    """
    Creates one hot encoded features per unique genre 

    """
    # Merges df and movies
    mini_df=df[df['userId']==userId]
    rating_films=pd.merge(movies, mini_df, on='movieId', how='inner')
    rating_films.drop(columns='movieId',inplace=True)
    # Create list of unique_genres
    unique_genres = movies['genres'].unique()
    all_genres = set('|'.join(unique_genres).split('|'))

    all_genres_list = list(all_genres)
    all_genres_list.remove('(no genres listed)')
    
    unique_director = movies['director'].unique()

    all_director_list = list(unique_director)
    all_director_list.remove('nobody')
    # Create a one hot encoding for each genre
    for genre in all_genres_list:
        rating_films[genre] = rating_films['genres'].apply(lambda x: genre in x.split('|'))
    # Create a one hot encoding for each director
    for director in all_director_list:
        rating_films[director] = rating_films['director'].apply(lambda x: director in x)
    rating_films.drop(columns=['director','genres','timestamp'],inplace=True)
    return rating_films

In [155]:
testing=Total_eval_test(100,df,test_merge)

/var/folders/bz/jj60x6y164ldxfdjhqsnv6dm0000gn/T/ipykernel_48654/1370888562.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rating_films[director] = rating_films['director'].apply(lambda x: director in x)
/var/folders/bz/jj60x6y164ldxfdjhqsnv6dm0000gn/T/ipykernel_48654/1370888562.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rating_films[director] = rating_films['director'].apply(lambda x: director in x)
/var/folders/bz/jj60x6y164ldxfdjhqsnv6dm0000gn/T/ipykernel_48654/1370888562.py:26: PerformanceWarning: DataFrame i

In [153]:
def evaluate_model_for_user_test(userId, data):
    """
    Generates training and test splits for a given user ID, trains a neural network,
    and evaluates the model to return MSE and R^2 for both training and test sets.
    """
    # Filter and preprocess data for the given user
    user_rating = data[data['userId'] == userId]
    user_rating = user_rating.sample(frac=1)  # Shuffle data
    y = user_rating['rating'].astype('float32')
    x = user_rating.drop(columns='rating').astype('float32')

    # Split into training and testing sets
    length = len(y)
    x_size = int(length * 0.8)
    x_train, x_test = x.iloc[:x_size, :], x.iloc[x_size:, :]
    y_train, y_test = y.iloc[:x_size], y.iloc[x_size:]

    # Define the neural network model
    model = Sequential([
        Dense(64, input_dim=x_train.shape[1], activation='relu'),  # First hidden layer
        Dense(32, activation='relu'),  # Second hidden layer
        Dense(1, activation='linear')  # Output layer for regression
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    # Train the model
    model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.3, verbose=1)

    # Predict on training and test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Calculate evaluation metrics
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # Print evaluation results
    print(f"Evaluation for User ID {userId}:")
    print("Training Set Evaluation:")
    print(f"MSE: {train_mse:.4f}, R^2: {train_r2:.4f}")
    print("\nTest Set Evaluation:")
    print(f"MSE: {test_mse:.4f}, R^2: {test_r2:.4f}")

    # Return MSE values for both sets
    return train_mse, test_mse


In [156]:
evaluate_model_for_user_test(100,testing)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 6.1557 - mse: 6.1557 - val_loss: 1.2965 - val_mse: 1.2965
Epoch 2/100
1/1 [==============================] - 0s 41ms/step - loss: 1.2492 - mse: 1.2492 - val_loss: 1.4060 - val_mse: 1.4060
Epoch 3/100
1/1 [==============================] - 0s 35ms/step - loss: 2.8411 - mse: 2.8411 - val_loss: 1.8527 - val_mse: 1.8527
Epoch 4/100
1/1 [==============================] - 0s 45ms/step - loss: 3.1875 - mse: 3.1875 - val_loss: 1.2349 - val_mse: 1.2349
Epoch 5/100
1/1 [==============================] - 0s 60ms/step - loss: 1.8902 - mse: 1.8902 - val_loss: 0.6659 - val_mse: 0.6659
Epoch 6/100
1/1 [==============================] - 0s 43ms/step - loss: 0.7842 - mse: 0.7842 - val_loss: 0.7930 - val_mse: 0.7930
Epoch 7/100
1/1 [==============================] - 0s 41ms/step - loss: 0.6391 - mse: 0.6391 - val_loss: 1.3349 - val_mse: 1.3349
Epoch 8/100
1/1 [==============================] - 0s 38ms/step - loss: 1.1881 - mse: 1.1881

(0.3628002, 0.65219486)

In [160]:
a=df_tmdb_keywords[df_tmdb_keywords['id']==862]
a

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
